In [1]:
import numpy as np
import plotly.graph_objects as go
import plotly.express as px

# ---------------------------
# Konstanter og parametere
# ---------------------------
c = 3e8  # m/s
frequency = 1e9      # Hz
sat_height = 50e3     # m
pixel_size = 10       # m (romlig oppløsning)
domain_size = 500     # m^2
wavelength = c / frequency

epsilon_regolith = 2.5
epsilon_ice = 3.15

crater_center = (0, 0)
crater_radius = 200
crater_depth = 400
mixing_depth = 20 # Avgjør fraksjonen av vannis

# ---------------------------
# 1. Grid og overflate
# ---------------------------
x = np.arange(-domain_size/2, domain_size/2, pixel_size)
y = np.arange(-domain_size/2, domain_size/2, pixel_size)
X, Y = np.meshgrid(x, y)

R = np.sqrt((X - crater_center[0])**2 + (Y - crater_center[1])**2)
surface = np.zeros_like(X)
inside_crater = R < crater_radius
surface[inside_crater] = -crater_depth * (1 - (R[inside_crater] / crater_radius)**2)

# Add some texture to the top of the crater with rounded edges and a small wall
terrain_texture = np.random.uniform(0, 20, size=surface.shape)
surface += terrain_texture

# ---------------------------
# 2. Vanniskuler i kraterets bunn
# ---------------------------
np.random.seed(69420)
num_ice_spheres = 80
ice_spheres = []

for _ in range(num_ice_spheres):
    radius = np.random.normal(15, 5)
    depth = np.random.uniform(1, 5)
    
    # 1. Velg en tilfeldig høyde (cz) for vanniskulen
    cz = np.random.normal(-crater_depth-radius, crater_depth / 4)

    # 2. Beregn den tilgjengelige radiusen på kraterveggen for denne høyden
    valid_indices = np.where(np.abs(surface - cz) < 1)  # Finn punktene der høyden cz er nærmest overflaten
    possible_R = R[valid_indices]  # Radius på de punktene

    if len(possible_R) > 0:
        # Velg en tilfeldig radius for isblokken fra mulige verdier
        R_at_cz = np.random.choice(possible_R) + radius*1.1

        # 3. Velg en tilfeldig posisjon (x, y) på kraterveggen ved høyden cz
        angle = np.random.uniform(0, 2 * np.pi)
        cx = R_at_cz * np.cos(angle)  # x-posisjon
        cy = R_at_cz * np.sin(angle)  # y-posisjon

        # 4. Legg kulen til listen
        ice_spheres.append((cx, cy, cz, radius))

# ---------------------------
# 3. Beregn vannisfraksjon i hvert piksel
# ---------------------------
f_ice = np.zeros_like(X, dtype=float)

for cx, cy, cz, radius in ice_spheres:
    dist_xy = np.sqrt((X - cx)**2 + (Y - cy)**2)
    depth_from_surface = surface - cz
    in_sphere = (dist_xy <= radius) & (depth_from_surface <= radius)

    f_block = np.clip(radius / mixing_depth, 0, 1)
    f_ice[in_sphere] = np.maximum(f_ice[in_sphere], f_block)

# ---------------------------
# 4. Effektiv permittivitet og SAR-refleksjon
# ---------------------------
epsilon_eff = f_ice * epsilon_ice + (1 - f_ice) * epsilon_regolith

r_horiz = np.sqrt(X**2 + Y**2)
d_surface = np.sqrt(r_horiz**2 + (sat_height - surface)**2)
theta = np.arctan2(r_horiz, (sat_height - surface))
sqrt_term = np.sqrt(np.maximum(epsilon_eff - np.sin(theta)**2, 0))
r_eff = (np.cos(theta) - sqrt_term) / (np.cos(theta) + sqrt_term)

# Inkluder bølgelengden i refleksjonskoeffisienten
r_eff = r_eff * np.exp(-2j * np.pi * d_surface / wavelength)
amp = np.abs(r_eff)

# ---------------------------
# 5. Plotly 3D-visualisering
# ---------------------------
fig3d = go.Figure()

fig3d.add_trace(go.Surface(
    z=surface, x=X, y=Y,
    colorscale=[[0, 'black'], [1, 'gray']],
    showscale=False, opacity=1.0,
    name="Overflate"
))

for cx, cy, cz, radius in ice_spheres:
    u = np.linspace(0, 2 * np.pi, 20)
    v = np.linspace(0, np.pi, 20)
    u, v = np.meshgrid(u, v)
    xs = radius * np.cos(u) * np.sin(v) + cx
    ys = radius * np.sin(u) * np.sin(v) + cy
    zs = radius * np.cos(v) + cz
    fig3d.add_trace(go.Surface(
        x=xs, y=ys, z=zs,
        colorscale=[[0, 'lightblue'], [1, 'lightblue']],
        showscale=False, opacity=0.7,
        name='Vannis'
    ))

fig3d.update_layout(
    width=800, height=800,
    title="3D-visualisering: Krater og vanniskuler",
    scene=dict(
        xaxis_title='x', yaxis_title='y', zaxis_title='z',
        xaxis=dict(backgroundcolor='black', gridcolor='gray', color='white'),
        yaxis=dict(backgroundcolor='black', gridcolor='gray', color='white'),
        zaxis=dict(backgroundcolor='black', gridcolor='gray', color='white'),
        bgcolor='black'
    ),
    paper_bgcolor='black',
    plot_bgcolor='black',
    font=dict(color='white')
)
fig3d.show()

# ---------------------------
# 6. Plotly 2D Visualisering med svart bakgrunn
# ---------------------------

fig_amp = px.imshow(amp, x=x, y=y, color_continuous_scale='gray',
                   origin='lower',
                   labels={'color': 'Refleksjonsamplitude'},
                   title='SAR-intensitet (refleksjonsamplitude)')
fig_amp.update_layout(
    width=800, height=800,
    xaxis_title='x',
    yaxis_title='y',
    paper_bgcolor='black',
    plot_bgcolor='black',
    font_color='white',
    coloraxis_colorbar=dict(bgcolor='black'),
    xaxis=dict(showgrid=True, gridcolor='white', zeroline=False, color='white'),
    yaxis=dict(showgrid=True, gridcolor='white', zeroline=False, color='white')
)
fig_amp.show()

fig_eps = px.imshow(epsilon_eff, x=x, y=y, color_continuous_scale='viridis',
                   origin='lower',
                   labels={'color': 'ε_eff'},
                   title='Effektiv permittivitet')
fig_eps.update_layout(
    width=800, height=800,
    xaxis_title='x',
    yaxis_title='y',
    paper_bgcolor='black',
    plot_bgcolor='black',
    font_color='white',
    coloraxis_colorbar=dict(bgcolor='black'),
    xaxis=dict(showgrid=True, gridcolor='white', zeroline=False, color='white'),
    yaxis=dict(showgrid=True, gridcolor='white', zeroline=False, color='white')
)
fig_eps.show()

fig_frac = px.imshow(f_ice, x=x, y=y, color_continuous_scale='Blues',
                    origin='lower',
                    labels={'color': 'Vannisfraksjon'},
                    title='Vannisfraksjon')
fig_frac.update_layout(
    width=800, height=800,
    xaxis_title='x',
    yaxis_title='y',
    paper_bgcolor='black',
    plot_bgcolor='black',
    font_color='white',
    coloraxis_colorbar=dict(bgcolor='black'),
    xaxis=dict(showgrid=True, gridcolor='white', zeroline=False, color='white'),
    yaxis=dict(showgrid=True, gridcolor='white', zeroline=False, color='white')
)
fig_frac.show()